# Generate ensembl gene_id and gene_name one-on-one annotation file
if gene_name has duplicate gene_id, will be gene_name:gene_id

In [1]:
import pandas as pd
import csv

In [2]:
'''
step1:
load gene annoataion file, gene id and gene name (one to one)
'''
####complete function, may need slight modification based on your gtf format####
def get_ens_dict(file_path):
    with open(file_path) as f:
        gtf = list(f)

    gtf = [x for x in gtf if not x.startswith('#')]
    gtf = [x for x in gtf if 'gene_id "' in x and 'gene_name "' in x]
    if len(gtf) == 0:
        print('you need to change gene_id " and gene_name " formats')
    
    gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('gene_name "')[1].split('"')[0]), gtf))
    gtf = dict(set(gtf))
    return gtf

gtf_dict = get_ens_dict('./Homo_sapiens.GRCh38.107.gtf')

In [3]:
df_an = pd.read_csv('./ensembl_mod_annotation.csv', skiprows = 0, sep = ',')

In [4]:
df_an["gene_name_add"] = df_an["gene_id"]
df_an = df_an.replace({"gene_name_add": gtf_dict})
df_an

,gene_id,gene_name,gene_name_add
0,ENSG00000000457,SCYL3,SCYL3
1,ENSG00000000460,C1orf112,C1orf112
2,ENSG00000000938,FGR,FGR
3,ENSG00000000971,CFH,CFH
4,ENSG00000001460,STPG1,STPG1
...,...,...,...
61855,ENSG00000288686,XGY2,XGY2
61856,ENSG00000289705,ENSG00000289705,ENSG00000289705
61857,ENSG00000289706,ENSG00000289706,ENSG00000289706
61858,ENSG00000289707,ENSG00000289707,ENSG00000289707


In [5]:
df_gene_name_count = df_an["gene_name_add"].value_counts().reset_index().rename(columns={"index": "gene_name_add", "gene_name_add": "count"})
df_gene_name_count

,gene_name_add,count
0,Y_RNA,758
1,Metazoa_SRP,169
2,U3,50
3,U6,37
4,SNORA70,27
...,...,...
60640,AARS1,1
60641,GLG1,1
60642,TNRC6A,1
60643,ABCC6,1


In [6]:
pd_all_gene_add_cnt = pd.merge(df_an, df_gene_name_count, left_on=["gene_name_add"], right_on=["gene_name_add"], how='outer')

In [7]:
def conditions(df):
    if (df['count'] == 1):
        return df['gene_name_add']
    elif ((df['count'] > 1) & (df["gene_name"] != df["gene_name_add"])):
        return df['gene_name_add'] + ":" + df['gene_name']
    else:
        return df['gene_name_add']

pd_all_gene_add_cnt["final_gene_name"] = pd_all_gene_add_cnt.apply(conditions, axis=1)
pd_all_gene_add_cnt

,gene_id,gene_name,gene_name_add,count,final_gene_name
0,ENSG00000000457,SCYL3,SCYL3,1,SCYL3
1,ENSG00000000460,C1orf112,C1orf112,1,C1orf112
2,ENSG00000000938,FGR,FGR,1,FGR
3,ENSG00000000971,CFH,CFH,1,CFH
4,ENSG00000001460,STPG1,STPG1,1,STPG1
...,...,...,...,...,...
61855,ENSG00000288686,XGY2,XGY2,1,XGY2
61856,ENSG00000289705,ENSG00000289705,ENSG00000289705,1,ENSG00000289705
61857,ENSG00000289706,ENSG00000289706,ENSG00000289706,1,ENSG00000289706
61858,ENSG00000289707,ENSG00000289707,ENSG00000289707,1,ENSG00000289707


In [10]:
pd_all_gene_add_cnt[pd_all_gene_add_cnt["count"] > 1]

,gene_id,gene_name,gene_name_add,count,final_gene_name
784,ENSG00000143248,ENSG00000143248,RGS5,2,RGS5:ENSG00000143248
785,ENSG00000232995,ENSG00000232995,RGS5,2,RGS5:ENSG00000232995
1867,ENSG00000199241,ENSG00000199241,Y_RNA,758,Y_RNA:ENSG00000199241
1868,ENSG00000199349,ENSG00000199349,Y_RNA,758,Y_RNA:ENSG00000199349
1869,ENSG00000199442,ENSG00000199442,Y_RNA,758,Y_RNA:ENSG00000199442
...,...,...,...,...,...
58540,ENSG00000280496,ENSG00000280496,SNORA17B,2,SNORA17B:ENSG00000280496
59346,ENSG00000158427,ENSG00000158427,TMSB15B,2,TMSB15B:ENSG00000158427
59347,ENSG00000269226,ENSG00000269226,TMSB15B,2,TMSB15B:ENSG00000269226
61009,ENSG00000271147,ENSG00000271147,ARMCX5-GPRASP2,2,ARMCX5-GPRASP2:ENSG00000271147


In [9]:
pd_all_gene_final = pd_all_gene_add_cnt[["gene_id", "final_gene_name"]].rename(columns={"final_gene_name": "gene_name"})
pd_all_gene_final

,gene_id,gene_name
0,ENSG00000000457,SCYL3
1,ENSG00000000460,C1orf112
2,ENSG00000000938,FGR
3,ENSG00000000971,CFH
4,ENSG00000001460,STPG1
...,...,...
61855,ENSG00000288686,XGY2
61856,ENSG00000289705,ENSG00000289705
61857,ENSG00000289706,ENSG00000289706
61858,ENSG00000289707,ENSG00000289707


In [13]:
pd_all_gene_final.to_csv("./ensembl_grch38_107_annotation.csv", index=False)

In [ ]:
# with open('/mnt/data/kailu/STAR_example/ensembl/ensembl_grch38_107_annotation.csv', 'w') as csv_file:  
#     writer = csv.writer(csv_file)
#     for key, value in gtf_dict.items():
#        writer.writerow([key, value])